In [43]:
# Model design
import agentpy as ap
import random

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython


In [44]:
# If agentpy is not installed, uncomment this
# ! pip install agentpy

In [45]:

'''
Rules of the system:
1. A burning tree becomes an empty site
2. A green tree becomes burning if at least one of its neighbours is burning
3. At an empty site, a tree grows with the probability p
4. A tree without a burning neighbour becomes a burning tree at any time step with the probability f ('lightning')

Modifications needed:
1. Do not automatically start the burning
2. Add a condition where a tree starts growing
3. Adjust probabilities
4. Do not stop the animation
Hello :)
'''

"\nRules of the system:\n1. A burning tree becomes an empty site\n2. A green tree becomes burning if at least one of its neighbours is burning\n3. At an empty site, a tree grows with the probability p\n4. A tree without a burning neighbour becomes a burning tree at any time step with the probability f ('lightning')\n\nModifications needed:\n1. Do not automatically start the burning\n2. Add a condition where a tree starts growing\n3. Adjust probabilities\n4. Do not stop the animation\nHello :)\n"

In [46]:
class ForestModel(ap.Model):

    def setup(self):

        # Create agents (trees)
        n_trees = int(self.p['Tree density'] * (self.p.size**2))
        trees = self.agents = ap.AgentList(self, n_trees)

        # Create grid (forest)
        self.forest = ap.Grid(self, [self.p.size]*2, track_empty=True)
        self.forest.add_agents(trees, random=True, empty=True)

        # Initiate a dynamic variable for all trees
        # Condition 0: Alive, 1: Burning, 2: Empty
        self.agents.condition = 0

        # Probability of lightning
        self.f = self.p['probability of lightning']

        # Probability of tree growth
        self.new = self.p['probability of new tree']

        # Start a fire from the left side of the grid
        # unfortunate_trees = self.forest.agents[0:self.p.size, 0:2]
        # unfortunate_trees.condition = 1

    def step(self):

        # Select burning trees
        burning_trees = self.agents.select(self.agents.condition == 1)

        # Select empty lots
        no_trees = self.agents.select(self.agents.condition == 2)

        # Select healthy trees
        healthy_trees = self.agents.select(self.agents.condition == 0)

        # Add lighting
        for tree in healthy_trees:
            if random.random() < self.f:
                tree.condition = 1

        # Spread fire
        for tree in burning_trees:
            for neighbor in self.forest.neighbors(tree):
                if neighbor.condition == 0:
                    neighbor.condition = 1 # Neighbor starts burning
            tree.condition = 2 # Tree burns out
        
        for tree in no_trees:
            if random.random() < self.new:
                tree.condition == 0

        # Stop simulation if no fire is left
       # if len(burning_trees) == 0:
        #    self.stop()

    def end(self):

        # Document a measure at the end of the simulation
        burned_trees = len(self.agents.select(self.agents.condition == 2))
        self.report('Percentage of burned trees',
                    burned_trees / len(self.agents))

In [47]:
# Define parameters

parameters = {
    'Tree density': 0.5, # Percentage of grid covered by trees
    'size': 50, # Height and length of the grid
    'steps': 100,
    'probability of lightning': 0.01,
    'probability of new tree': 0.05
}

In [48]:
# Create single-run animation with custom colors

def animation_plot(model, ax):
    attr_grid = model.forest.attr_grid('condition')
    color_dict = {0:'#7FC97F', 1:'#d62c2c', 2:'#e5e5e5', None:'#d5e5d5'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulation of a forest fire\n"
                 f"Time-step: {model.t}, Trees left: "
                 f"{len(model.agents.select(model.agents.condition == 0))}")

fig, ax = plt.subplots()
model = ForestModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))